# Chapter 6: Decision Trees

São algorítimos versáteis que podem fazer classificação e regressão e até tarefas multioutputs. É o modelo base e fundamental das florestas, que são muito poderosas.

## Treinando e visualizando uma árvore de decisão

Para entender árvore de decisão vamos construir uma e olhar a forma como ela faz predições.

In [17]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()

X = iris.data[:, 2:] # petal length and width
y = iris.target


In [18]:
# visualize the data and target
list(zip(X, y))

[(array([1.4, 0.2]), 0),
 (array([1.4, 0.2]), 0),
 (array([1.3, 0.2]), 0),
 (array([1.5, 0.2]), 0),
 (array([1.4, 0.2]), 0),
 (array([1.7, 0.4]), 0),
 (array([1.4, 0.3]), 0),
 (array([1.5, 0.2]), 0),
 (array([1.4, 0.2]), 0),
 (array([1.5, 0.1]), 0),
 (array([1.5, 0.2]), 0),
 (array([1.6, 0.2]), 0),
 (array([1.4, 0.1]), 0),
 (array([1.1, 0.1]), 0),
 (array([1.2, 0.2]), 0),
 (array([1.5, 0.4]), 0),
 (array([1.3, 0.4]), 0),
 (array([1.4, 0.3]), 0),
 (array([1.7, 0.3]), 0),
 (array([1.5, 0.3]), 0),
 (array([1.7, 0.2]), 0),
 (array([1.5, 0.4]), 0),
 (array([1. , 0.2]), 0),
 (array([1.7, 0.5]), 0),
 (array([1.9, 0.2]), 0),
 (array([1.6, 0.2]), 0),
 (array([1.6, 0.4]), 0),
 (array([1.5, 0.2]), 0),
 (array([1.4, 0.2]), 0),
 (array([1.6, 0.2]), 0),
 (array([1.6, 0.2]), 0),
 (array([1.5, 0.4]), 0),
 (array([1.5, 0.1]), 0),
 (array([1.4, 0.2]), 0),
 (array([1.5, 0.2]), 0),
 (array([1.2, 0.2]), 0),
 (array([1.3, 0.2]), 0),
 (array([1.4, 0.1]), 0),
 (array([1.3, 0.2]), 0),
 (array([1.5, 0.2]), 0),


treinando

In [19]:

tree_clf = DecisionTreeClassifier(max_depth = 3)
tree_clf.fit(X, y)
pass

exportando desenho da árvore

In [20]:
from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file = 'iris_tree.dot',
    feature_names = iris.feature_names[2:],
    class_names = iris.target_names,
    rounded = True,
    filled = True
)

# comando bash:
# dot -Tpng iris_tree.dot -o iris_tree.png
# olhar iris_tree.png

## Previsões

As previsões em árvores são feitas conforme o esquemático mostrado acima. Cada nó faz uma pergunta e bifurca para outros nós a depender da resposta.

Uma vantagem das árvores é que elas precisam de pouca preparação dos dados e não se importam com feature scaling ou feature centering.

O atributo 'samples' dos nós contam quantas instâncias chegaram nesse nó.

O atributo 'gini' mede a impureza do nó. Se gini = 0, todas as instâncias pertencem a mesma classe. Veja a fórmula do gini:

$$
G_i = 1 - \sum_{k=1}^{n}p_{i,k}^2
$$

* $p_{i,k}$ é a porcentagem de elementos da classe k nos samples do i-ésimo nó

suponha que vc tem 3 classes possíveis e em um nó temos 54 samples. Sendo (classe1: 0, classe2: 49, classe3: 5). Assim, a impureza é de

 $1 - (\frac{0}{54})^2 - (\frac{49}{54})^2- (\frac{5}{54})^2 \approx 0.168$

O sklearn usa o algoritmo CART que produz apenas árvores binárias. Outros algoritmos como ID3 podem produzir árvores não binárias.

Se a produndidade fosse até 3, as linhas verticais pontilhadas seriam feitas.

In [21]:
from mlxtend.plotting import plot_decision_regions

![](./imgs/treeboundary.png)

Árvores são modelos transparentes, 'white box'. Diferentemente das florestas e das redes neurais.

## Estimando as probabilidades das classes

Para uma instância, a árvore primeiro tenta encaixá-la em uma folha e, em seguida, cospe sua probabilidade baseada na pureza dessa folha nos dados de treino.

usando o exemplo anterior: uma flor com 5cm x 1.5cm de dimensão é colocada no nó da esquerda de profundidade 2. Assim, usando o exemplo (0, 49, 5), a árvore vai dizer que essa instância tem 49/54 % de chance de ser Versicolor (classe2) e 5/54 % de chance de ser virginica.

In [22]:
tree_clf.predict_proba([[5, 1.5]])

array([[0.        , 0.33333333, 0.66666667]])

## O algoritmo CART

O algoritmo CART (classification And Regression Tree) é usado para treinar as árvores de decisão. A ideia é simples: o algoritmo separa o dataset de treino em 2 subconjuntos usando uma única feature $k$ e um threshold $t_k$. Como ele escolhe $k$ e $t_k$? Ele busca pelo par $(k, t_k)$ que produz os subconjuntos mais puros (usando o tamanho como peso). A função de custo que o algoritmo tenta minimizar é dada por:

$$
J(k, t_k) = \frac{m_{left}}{m}G_{left} + \frac{m_{right}}{m}G_{right}
$$

* $G_{left/right}$ mede a impureza do subconjunto esquerdo/direito
* $m_{left/right}$ é o número de instâncias no subconjunto da esquerda/direita

Uma vez que se consegue fazer isso, ela separa os subconjuntos de novo usando a mesma lógica. Isso para quando chega a profundidade máxima (hyperparameter) ou se não consegue encontrar uma divisão que reduza impureza.

Percebam que CART é um algoritmo greedy/guloso. Ele procura por uma divisão ótima a cada profundidade, sem saber se isso vai resultar em impurezas pequenas em maiores profundidades. Não há garantia de ser uma solução ótima.

Há como encontrar a árvore perfeita! Pena que a ordem complexidade é $O(exp(m))$.

## Complexidade

Predição: $O(log_2(m))$, muito rápido. Independe do número de features(n)

Treino: $O(n\times mlog(m))$. Ele compara todas as features em todos os samples em cada nó (a menos que max_features seja determinado).

## Gini impurity or Entropy?

O default é o Gini, mas vc pode selecionar a entropia. (criterion = 'entropy'). Entropia para o i-ésimo nó é dada por:

$$
H_i = - \sum_{k=1}^n p_{i,k} log(p_{i, k})
$$

Qual usar? tanto faz. As diferenças são ínfimas. Mas gini tende a ser mais rápido e entropia a deixar a arvore mais balanceada.

## Hiperparâmetros de regularização

Árvore assume pouca coisa sobre os dados e, por isso, se deixá-la sem constraints, ela vai overfittar.

max_depth -> profundidade máxima da árvore.

min_samples_split -> número de instâncias um nó deve ter antes de se quebrar

min_samples_leaf -> número mínimo de instâncias que uma folha deve ter.

min_weight_fraction_leaf -> mesmo do de cima mas expresso como uma fração do total de instancias

max_leaf_nodes -> máximo de folhas.

max_features -> máximo de features que são observadas para splittar em cada nó

regularizar: aumentar min's ou diminuir max's

Alguns algoritmos podem gerar uma árvore sem regularização e, em seguida, podá-la, retirando nós que não trazem ganhos reais.

exemplo:

![](./imgs/treeover.png)

## Regressão

Árvores podem fazer regressão e, para isso, elas determinam um valor para cada região no nosso espaço de features.

exemplo:

![](./imgs/treereg.png)

Continuamos usando o CART mas ao invés de minimizar impureza minimizamos o MSE. Função de custo:

$$
J(k, t_k) = \frac{m_{left}}{m}MSE_{left} + \frac{m_{right}}{m}MSE_{right}
$$

exemplo de regressão:

![](./imgs/treereg2.png)

## Instabilidade

Percebemos que árvores de decisão amam decision boundaries ortogonais. Isso as faz muito sensíveis a rotação dos dados.

exemplo:

![](./imgs/treerot.png)

Uma forma de melhorar nesse quesito é usar PCA, que frequentemente resulta em uma orientação melhor dos dados.

## Exercícios

1. Qual a profundidade de uma árvore de decisão treinada sem restrições sobre um dataset de 1M de instâncias?

r: $log_2(10^6)$

2. O gini de um nó filho é maior ou menor que o do pai?

r: normalmente menor, mas a função de custo visa diminuir a soma ponderada. Portanto, um filho pode ter gini mais alto que o do pai se o outro tiver gini baixo.

3. Diminuir max_depth ajuda contra overfitting?

r: sim, é uma forma de regularizar!

4. Se uma árvore está underfittando, faz sentido padronizar as features?

r: Não. Em geral não surte muito efeito.

5. qual a complexidade do treino?

r: $O(n \times mlog(m))$

6. Com 100k linhas, presort = True irá acelerar o treino?

r: Não. Só acelera para datasets pequenos (<10k)